In [514]:
import cv2 as cv
from pyzbar import pyzbar
import matplotlib.pyplot as plt
import numpy as np

In [515]:
def detect_qr_from_camera():
    cap = cv.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Không thể đọc khung hình từ camera.")
            break

        qrCodes = pyzbar.decode(frame)
        for qrCode in qrCodes:
            (x, y, w, h) = qrCode.rect
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)
            qrCodeData = qrCode.data.decode("utf-8")
            qrCodeType = qrCode.type
            text = "{} - {}".format(qrCodeData, qrCodeType)
            print(text)
            cv.putText(frame, text, (x-10, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        cv.imshow("Camera", frame)
        if cv.waitKey(1) == ord('q'):
            break

    cap.release()
    cv.destroyAllWindows()

In [516]:
def gamma(img):
    # Tính toán giá trị trung bình của ảnh
    mean_intensity = np.mean(img)
    
    # Xác định giá trị gamma phù hợp
    if mean_intensity < 100:
        g = 0.5  # Đối với ảnh quá tối, sử dụng gamma = 0.5 để tăng độ sáng
    elif mean_intensity > 200:
        g = 2.0  # Đối với ảnh quá sáng, sử dụng gamma = 2.0 để giảm độ sáng
    else:
        g = 1.0  # Đối với ảnh có độ sáng trung bình, không cần thay đổi
    
    # Áp dụng bộ lọc gamma với giá trị g
    newImg = np.power(img/255, g) * 255
    
    return newImg


In [517]:
def gamma_check(img):
    # Tính toán giá trị trung bình của ảnh
    mean_intensity = np.mean(img)
    
    # Xác định giá trị gamma phù hợp
    if mean_intensity < 100:
        g = 0.5  # Đối với ảnh quá tối, sử dụng gamma = 0.5 để tăng độ sáng
    elif mean_intensity > 200:
        g = 2.0  # Đối với ảnh quá sáng, sử dụng gamma = 2.0 để giảm độ sáng
    else:
        g = 1.0  # Đối với ảnh có độ sáng trung bình, không cần thay đổi
    
    # Áp dụng bộ lọc gamma với giá trị g
    newImg = np.power(img/255, g) * 255
    
    return mean_intensity


In [518]:
# Định nghĩa bộ lọc 3x3 trung vị của ảnh
def medianImg3x3(img):
    m, n = img.shape
    img_new = np.zeros_like(img)
    for i in range(1, m-1):
        for j in range(1, n-1):
            temp = [img[i-1, j-1], img[i-1, j], img[i-1, j + 1],
                    img[i, j-1], img[i, j], img[i, j + 1],
                    img[i + 1, j-1], img[i + 1, j], img[i + 1, j + 1]]

            temp = sorted(temp)
            img_new[i, j] = temp[4]

    return img_new

In [519]:
def histogram_equalization(img):
    # Tính histogram của ảnh
    hist, _ = np.histogram(img.flatten(), 256, [0,256])

    # Tính toán hàm tích lũy của histogram
    cdf = hist.cumsum()

    # Chuẩn hóa hàm tích lũy về khoảng [0,255]
    cdf_normalized = cdf * hist.max() / cdf.max()

    # Tính toán giá trị pixel mới dựa trên hàm tích lũy chuẩn hóa
    img_equalized = np.interp(img.flatten(), range(256), cdf_normalized).reshape(img.shape)

    return img_equalized.astype(np.uint8)

In [520]:
def view(qrCodes_processed, image, img):    
    for qrCode in qrCodes_processed:
        (x, y, w, h) = qrCode.rect
        cv.rectangle(image, (x, y), (x+w, y+h), (0, 255, 255), 2)
        qrCodeData = qrCode.data.decode("utf-8")
        qrCodeType = qrCode.type
        text = "{} - {}".format(qrCodeData, qrCodeType)
        print(text)
        cv.putText(image, text, (x-10, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    cv.imshow("Phat hien QR code", image)  
    cv.imshow("Anh goc", img) 

    key = cv.waitKey(0)

    if key == ord('q'):
        cv.destroyAllWindows()


In [521]:
def detect_qr_from_image(image_path):
    image = cv.imread(image_path)
    if image is None:
        print("Không thể đọc ảnh từ đường dẫn được cung cấp.")
        return

    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    qrCodes = pyzbar.decode(image)

    if len(qrCodes) == 0:
        print("Không tìm thấy mã QR trong ảnh.")

        clahe = cv.createCLAHE(clipLimit=15.0, tileGridSize=(35, 35))
        equalized_gray = clahe.apply(gray)
        qrCodes = pyzbar.decode(equalized_gray)
        if len(qrCodes) == 0:
            print("Không phát hiện được mã QR trên ảnh sau khi được xử lý bằng cân bằng histogram")
            # Xử lý ảnh bằng medianImg3x3
            processed_image = medianImg3x3(equalized_gray)
            # Phát hiện mã QR trên ảnh đã được làm mờ
            qrCodes = pyzbar.decode(processed_image)

            if len(qrCodes) == 0:
                print("Không phát hiện được mã QR trên ảnh sau khi được xử lý bằng cân bằng trung vị")
            else:
                view(qrCodes, processed_image,image)
                
        else:
            view(qrCodes, equalized_gray, image)
            

    else:
        view(qrCodes, image, image)


In [522]:
detect_qr_from_image("./Img/case9.png")

Không tìm thấy mã QR trong ảnh.
https://qrco.de/bbyAiy - QRCODE


In [523]:
# detect_qr_from_camera()